Background sample : 각 class 별 10%

street-facilities-2  
class 0, 1, 2 (the counts of each object class, not image counts)  
[2698 7672 5397]  

X4, X1.5 X2  

10792, 11508, 10794  

X8, X3 X4  
21584, 23016, 21588  

### Image rotation

### 지금은 단순 Rotation 이긴 한데, 추후 수정 혹은 다른 증강방식을 적용 필요


In [ ]:
import os
import numpy as np
import cv2
import albumentations as A
import matplotlib.pyplot as plt
import warnings # to ignore np.loadtxt warning

root_path = "D:/Downloads/street-facilities-selected"
image_path = os.path.join(root_path, "images")
label_path = os.path.join(root_path, "labels-txt")

save_image_path = os.path.join(root_path, "rotated-images")
save_label_path = label_path

# image_names = [f for f in os.listdir(image_path) 
#                  if f.endswith('.jpg') or f.endswith('.jpeg')] # .jpeg 도 극히 일부 존재

# test
image_names = ["3_12_1_1_4_1_20210908_0000529500.jpg"]

total_cnt = len(image_names)

# ex "3_12_1_1_4_1_20210908_0000529500.jpg" -> the class type is 12
type_factor = {
    9 : 7, # type 9 -> 증강 7회 수행 (X8)
    12: 2, # (X3)
    13 : 3, # (X4)
}

# Albumentations Rotate 변환 설정
transform = A.Rotate(limit=45, # +- 45 degrees. Range from which a random angle is picked.
                    border_mode=cv2.BORDER_REPLICATE, 
                    p=1.0 # Probability of applying the transform. Default: 0.5.
                    )

for image_name in image_names:
    # 이미지 로드
    image = cv2.imread(os.path.join(image_path, image_name))
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    f_base = os.path.splitext(image_name)[0]
    class_type = int(f_base.split("_")[1]) # [9 or 12 or 13]
    label_file = os.path.join(label_path, f_base + ".txt")

    # 빈 txt 파일일 경우 뜨는 numpy 경고문 무시
    with warnings.catch_warnings():
        warnings.simplefilter("ignore") 
        bboxes = np.loadtxt(fname=label_file, delimiter=" ", ndmin=2)

    # albumentations 처리 형태로 변환
    bboxes = np.roll(bboxes, 4, axis=1)

    # 변환 적용
    # type 마다 반복 수 다름 
    for iter in range(type_factor[class_type]):
        # 변환
        transformed = transform(image=image, bboxes=bboxes)
        # 변환 결과
        rotated_image = transformed["image"]
        rotated_bboxes = transformed['bboxes']

        # albumentations 처리 형태에서 yolo label 형태로 다시 변환
        rotated_label = np.roll(rotated_bboxes, 1, axis=1)

        # 결과 이미지 시각화
        # plt.imshow(rotated_image)
        # plt.axis("off")
        # plt.show()

        # print(rotated_label)

        # cv2.imwrite(os.path.join(save_image_path, f_base + f"-{iter+1}.jpg"), rotated_image)
        # np.savetxt(os.path.join(save_label_path, f_base + f"-{iter+1}.txt"), rotated_label, delimiter=" ", fmt='%.8f')

        # test
        cv2.imwrite(f_base + f"-{iter+1}.jpg", rotated_image)
        np.savetxt(f_base + f"-{iter+1}.txt", rotated_label, delimiter=" ", fmt='%.8f')

    break

classId 는 Int 형이여야 하므로 이에 대한 처리 수행

In [ ]:
import os

root_path = "D:/Downloads/street-facilities-selected"

label_path = f"{root_path}/labels-txt"

labels = [label for label in os.listdir(label_path) if label.endswith('.txt')]
total_cnt = len(labels)

for i, label in enumerate(labels):
    if i % 1000 == 0 :
        print(f"{i+1} / {total_cnt}")
    annotations = []
    with open(os.path.join(label_path, label), "r+", encoding="utf-8") as f:
        # 기존 내용 읽기 및 수정
        for line in f:
            elems = line.split(" ")
            elems[0] = str(int(float(elems[0]))) # 1.00000000 -> 1 -> '1'
            newline = " ".join(elems)
            annotations.append(newline)
            
        # 새 내용 작성
        f.seek(0) # 파일 포인터 이동
        for annotation in annotations:
            f.write(annotation)
        f.truncate() # 파일 포인터 이전 내용만 남기고 나머지 삭제